In [2]:
from datasets import load_dataset
import ast

dataset = load_dataset("pnadel/michgovparsed8_16")
df = dataset['train'].to_pandas()
for col in ['From', 'Sent', 'To', 'Cc', 'Subject', 'Attachment']:
    df[col] = df[col].apply(ast.literal_eval)
df

Found cached dataset parquet (/Users/pnadel01/.cache/huggingface/datasets/pnadel___parquet/pnadel--michgovparsed8_16-9af338568c5dfeeb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

,From,Sent,To,Cc,Subject,Attachment,Body,org_file,formattedSent
0,"[(Scott, Allison, 6, 26)]","[(Tuesday, February 17, 2015 7:07 PM, 33, 67)]","[(Emmitt, Beth, 72, 90)]","[(Wisniewski, Wendy, 95, 118)]","[(Fwd: Tues meeting, 128, 145)]",[],Let's see if there is time on thurs that work...,Staff_10_djvu.txt,2015-02-17 19:07:00+00:00
1,"[(Snyder, Rick, 1, 12)]","[(February 17, 2015 at 1 1: 10:28 AM EST, 19, ...","[(Scott, Allison, 62, 75)]",[],"[(Re: Tues meeting, 85, 101)]",[],Thanks. Just got a text from US re dinner ton...,Staff_10_djvu.txt,2015-02-17 11:10:28+00:00
2,"[(Scott, Allison, 20, 40)]","[(Thursday, April 18, 2013 12:46 PM, 41, 74)]","[(Wisniewski, Wendy, 75, 98)]",[],"[(Fwd: COS items, 99, 113)]",[],Sent from my iPhone Begin forwarded message:,Staff_10_djvu.txt,2013-04-18 12:46:00+00:00
3,"[(Snyder, Rick, 2, 20)]","[(April 18, 2013, 12:29:57 PM EDT, 28, 59)]","[(Scott, Allison, 65, 85)]",[],"[(COS items, 127, 136)]","[(Flint/DWSD, 157, 167)]",Thanks Rick l,Staff_10_djvu.txt,2013-04-18 12:29:57+00:00
4,"[(Zdrodowski, Michelle, 1, 20)]","[(T uesday, October 1 3, 201 5 1 1 :24 AM, 64,...","[(Connersp@Michigan.Gov, 108, 129), (Muchmore,...","[(Rossman-Mckinney, Kelly, 363, 385), (Earley,...","[(Fw: Media Call:, 411, 426)]",[],Ron Fonger- Flint Journal (Deadline ASAP)-D. ...,Staff_10_djvu.txt,2015-10-13 11:24:00+00:00
...,...,...,...,...,...,...,...,...,...
5928,"[(Adamini, Dan, 6, 17)]","[(Tue, Dec 29, 2015 12:08 am, 24, 50)]",[],[],"[(Gov. Snyder aniel Adamini, 60, 85)]",[],"Hey, Dan! Hope you're going well; Merry Chris...",Staff_17_djvu.txt,2015-12-29 00:08:00+00:00
5929,"[(Nyberg, David, 1, 20)]",[],[],[],[(RE: NEWS UPDATE --Wyant Resigns In Wake Of F...,[],remember the PBB issue from one of my environ...,Staff_17_djvu.txt,NaT
5930,"[(Morals, Betsy, 1, 13)]","[(Tuesday, December 29, 2015 3:33 PM, 52, 86)]",[],[],[(FW; NEWS UPDATE--Wyant Resigns In Wake Of Fl...,[],Begin forwarded message:,Staff_17_djvu.txt,2015-12-29 15:33:00+00:00
5931,"[(News, Gongwer, 1, 21)]","[(December 29, 2015, 2:54:21 PM EST, 65, 98), ...","[(, 294, 382), (Scott, Allison, 3105, 3125), (...","[(Wisniewski, Wendy, 3346, 3369), (Saunders, K...",[(NEWS UPDATE-- Wyant Resigns In Wake Of Flint...,[],Reply-To: < gongwerreports@gongwer.com > Tues...,Staff_17_djvu.txt,2015-12-29 02:54:21+00:00


In [4]:
from nltk import sent_tokenize
import re
df['sents'] = df.Body.apply(lambda x: [s for s in sent_tokenize(x) if (len(re.findall('\t',s))<2) and (len(s.split(' '))>4)])

In [14]:
sents = df.reset_index().drop(['From', 'Body', 'To', 'Cc', 'Subject', 'Attachment', 'org_file', 'formattedSent', 'Sent'], axis=1).explode('sents').reset_index(drop=True).rename(columns={'index':'org_idx'})
data = sents.sents.dropna()

In [36]:
from txtai.embeddings import Embeddings
embeddings = Embeddings({"path": "sentence-transformers/msmarco-bert-base-dot-v5"})

In [37]:
embeddings.index([(uid, text, None) for uid, text in enumerate(data)])

In [35]:
uid = embeddings.search('water', limit=5)[4][0]
data[uid]

'Michael Prysby of the Michigan DEQ Office of Drinking Water verified that “the quality of the water being put out meets all of our drinking water standards and Flint water is safe to drink.” For nearly 10 years Mike Glasgow has worked in the laboratory at the City of Flint Water Service Center.'

In [19]:
embeddings.save('./data/fl_index')

In [18]:
sents.to_csv('./data/fl_sents.csv')

In [7]:
import pandas as pd
from ast import literal_eval

metadata = pd.read_csv('./data/lookup_data_for_threads.csv')
for col in ['To', 'Cc']:
    metadata[col] = metadata[col].apply(literal_eval)
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,0,0,Subject: Copy: City of Flint Drinking Water Is...,Staff_11_0000.txt,NaN,NaN,"[Brader; Valerie (GOV), Busch, Stephen (DEQ), ...",[],Copy: City of Flint Drinking Water Issues Loca...,CH-6S-DEQ-DIRECTOR,Update: This meeting has been confirmed for Mo...
1,1,1,Subject: City of Flint Drinking Water Issues L...,Staff_11_0001.txt,NaN,NaN,"[Brader; Valerie (GOV), Busch, Stephen (DEQ), ...",[],City of Flint Drinking Water Issues Location,CH-6S-DEQ-DIRECTOR,Update: This meeting has been confirmed for Mo...
2,2,2,NaN,Staff_11_0002.txt,NaN,NaN,[],[],NaN,NaN,NaN
3,3,2,"From: Wisniewski, Wendy (GOV) Sent: Wednesday;...",Staff_11_0002.txt,"Wisniewski, Wendy (GOV)","Wednesday; April 29, 2015 1.25 PM",[West; Samantha (GOV)],[],FW: Governor Snyder's Termination Letter,Flint EM Termination Letter pdf,NaN
4,4,2,"From: Wisniewski, Wendy (GOV) Sent: Wednesday,...",Staff_11_0002.txt,"Wisniewski, Wendy (GOV)","Wednesday, April 29, 2015 12.08 PM","[Stanton, Terry A. (Treasury), Storberg, Ann M...","[Koryzno, Edward (Treasury), Wisniewski, Wendy...",FW: Governor Snyder's Termination Letter,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30105,30105,16920,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1388.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."
30106,30106,16921,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1389.txt,NaN,NaN,[],[],NaN,City of Flint; Michigan Page 8 Printed on,4/2/2013
30107,30107,16922,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1390.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."
30108,30108,16923,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1391.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."


In [8]:
import pickle
with open('./data/email_threads.p', 'rb') as f:
        email_threads = pickle.load(f)

email_threads[2]

['Staff_17_0002.txt']

In [9]:
meta_threads = []
for t in email_threads:
    t_df = []
    for e in t:
        t_df.append(metadata.loc[metadata.image_lookup == e])
    t_df = pd.concat(t_df)
    meta_threads.append(t_df)

In [10]:
meta_threads[-4]

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
7832,7832,4908,NaN,Staff_7_0232.txt,NaN,NaN,[],[],NaN,NaN,NaN
7833,7833,4908,"From: Clayton; Stacie (GOV) Sent: Friday, Marc...",Staff_7_0232.txt,Clayton; Stacie (GOV),"Friday, March 06, 2015 10.34 AM",[Steve Linder],[],RE: Flint Water Meeting,NaN,Thank you: We will have about 8 people in atte...
7834,7834,4908,From: Steve Linder [mailto slinder@sterlingcor...,Staff_7_0232.txt,Steve Linder,"Friday, March 06, 2015 8.02 AM","[Clayton, Stacie (GOV)]",[],Re: Flint Water Meeting,NaN,Here is a brief sheet for them to review. We w...
7835,7835,4908,"From: <Clayton> , ""Stacie (GOV)' <claytons @mi...",Staff_7_0232.txt,"<Clayton> , ""","Thursday, March 5, 2015 12.47 PM",[Steve Linder <slinder@sterlingcorporation com>],[],RE: Flint Water Meeting,NaN,Thank you: Please send the information when yo...


In [99]:
import numpy as np
import re
actors = [n for n in list(meta_threads[-4].From) if (n is not np.nan) and (len(re.findall('[A-z]+',n))>=2)]
actors

['Clayton; Stacie (GOV)', 'Steve Linder']

In [96]:
prefix = ['Re', 'RE', 'FW', 'Fwd']

subjects = []
for s in set(meta_threads[-4].Subject):
    if s is not np.nan:
        if any([p in s for p in prefix]):
            subjects.append(' '.join(s.split()[1:]))
        else:
            subjects.append(s)

subjects = list(set(subjects))
subjects

['Flint Water Meeting']

In [11]:
meta_threads[0]

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
22427,22427,12184,Appointment,Staff_17_0000.txt,NaN,NaN,[],[],NaN,NaN,ppointment
22428,22428,12184,"From: Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHA...",Staff_17_0000.txt,"Rospond, Laurie (GOV)",1/13/2015 3.32:19 PM,"[Posthumus, Dick (GOV), Saxton, Thomas (Treasu...",[],Copy: Mtg Flint Water Issues/other Concerns Lo...,NaN,Attending= Jerry Ambrose Mayor Wa] Ting Tom Sa...


In [34]:
import numpy as np
threads_by_email = []
bodies_by_email = []
for mt in meta_threads:
    single_email = []
    single_body = []
    for i in range(len(mt)):
        if mt.iloc[i].From is not np.nan: # might want to change this
            actors = mt.iloc[i].To + mt.iloc[i].Cc
            actors = actors[:3] + [f'and {len(actors)-3} others'] if len(actors) > 4 else actors
            actors = ' '.join(actors)
            formatted_string = f"Email from {mt.iloc[i].From} to {actors}"
            formatted_string = formatted_string + f", concerning {mt.iloc[i].Subject}" if mt.iloc[i].Subject is not np.nan else formatted_string
            formatted_string = formatted_string + f" on {mt.iloc[i].Sent}" if mt.iloc[i].Sent is not np.nan else formatted_string
            single_email.append(formatted_string)
            single_email = list(pd.Series(single_email).str.wrap(70).apply(lambda x: x.replace('\n', '<br>')))

            single_body.append(mt.iloc[i].Body)
    if len(single_email) > 1: # might want to change this
        threads_by_email.append(single_email)
    
    if len(single_body) > 1:
        bodies_by_email.append(single_body)

In [35]:
threads_by_email # losing a lot here, but might be fine

[['Email from Brader;, Valerie (GOV) to Ackerman, Darin (GOV),<br>concerning<br>FW: Phone Msg: Senator Ananich on Tuesday, September 23,<br>2014 2.47 PM',
  'Email from West; Samantha (GOV) to Brader , Valerie (GOV), concerning<br>Phone Msg: Senator Ananich on Tuesday, September 23, 2014 2.43 PM'],
 ['Email from Starr; Scott (DTMB) to Ackerman, Darin<br>(GOV),<br>concerning<br>RE: Flint Water $ Breakdown on Friday, October<br>09, 2015<br>8.53 AM',
  'Email from Ackerman; Darin (GOV) to Starr, Scott (DTMB),<br>concerning<br>RE: Flint Water $ Breakdown on Friday, October 09, 2015<br>8.47 AM',
  'Email from Starr, Scott (DTMB) to Ackerman, Darin (GOV) Lange,<br>Michelle (GOV), concerning Flint Water $ Breakdown on Thursday,<br>October 08, 2015 2:12 PM'],
 ['Email from Shaquila Myers to Ackerman, Darin (GOV), concerning<br>RE:<br>Flint Water Funding Source on Friday, October 09, 2015 10.03 AM',
  'Email from Ackerman, Darin (GOV) to Andrew Leavitt, concerning Flint<br>Water Funding Source 

In [24]:
bodies_by_email

[['Uh????\n',
  'Called this afternoon: He was speaking with Rodney who advised that he should talk with you re water issues in the City of Flint: He can be reached at\nLet me know if you would like a call scheduled or if you want to just call him when you have a moment:'],
 ['Good morning,\nhave not shared that particular docurent with his office, but the fiscal agencies have it-~so no reason for you notto share ityou want Also, the formal supplemental request went to the Legislature (including his office} Iate yesterday afternoon, and that would inciude the breakdown as well, along with more detail than this sheet\n',
  "Do you know if Ananich's office has this yet? Ive only sent it to Bob and Pete so far since they asked:\nDarin Ackerman, Deputy Director Legislative Affairs Division Office of Governor Snyder State Capitol Lansing, MI 48913 Phone: (517) 241-3950\n",
  'Darin,\nPer your request, attached is a breakdown on the Flint water funding request:'],
 ['Thanks Darin_ 1 did not 

In [25]:
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,0,0,Appointment,Staff_17_0000.txt,NaN,NaN,[],[],NaN,NaN,ppointment
1,1,0,"From: Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHA...",Staff_17_0000.txt,"Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHANGE AD...","1/13/2015 3.32:19 PM To: Posthumus, Dick (GOV)...","['Pleyte, Beth (Treasury) [O-MIGOV/OU-EXTernal...",[],Copy: Mtg Flint Water Issues/other Concerns Lo...,1/28/2015 2.00.00 PM End: 1/28/2015 3.00.00 PM...,Attending= Jerry Ambrose Mayor Wa] Ting Tom Sa...
2,2,1,NaN,Staff_17_0001.txt,NaN,NaN,[],[],NaN,NaN,NaN
3,3,1,"From: Brader;, Valerie (GOV) Sent: Tuesday, Se...",Staff_17_0001.txt,NaN,"Tuesday, September 23, 2014 2.47 PM To: Ackerm...",[],[],NaN,NaN,Uh????\n
4,4,1,"From: West; Samantha (GOV) Sent: Tuesday, Sept...",Staff_17_0001.txt,NaN,"Tuesday, September 23, 2014 2.43 PM","['Brader , Valerie (GOV)']",[],Phone Msg: Senator Ananich,NaN,Called this afternoon: He was speaking with Ro...
...,...,...,...,...,...,...,...,...,...,...,...
30105,30105,16923,NaN,Staff_12_1359.txt,NaN,NaN,[],[],NaN,NaN,NaN
30106,30106,16923,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1359.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 12.42 PM","['Egan, Paul (pegan@freepresscom)']",[],Correct statement,NaN,"Paul, accidently sentyouthe outdated statement..."
30107,30107,16924,NaN,Staff_12_1360.txt,NaN,NaN,[],[],NaN,NaN,NaN
30108,30108,16924,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1360.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 1.09 PM","[""'Hollins, Harvey (GOV)""]",[],RE: Flint Talking Point,NaN,"Hello Harvey, after consulting with Jim Redfor..."


In [15]:
pickle.dump(threads_by_email, open('./data/threads_by_email.p', 'wb'))

In [199]:
import networkx as nx
longThreads = []
for ethread in threads_by_email:
    if len(ethread) > 3:
        longThreads.append(ethread)

G = nx.DiGraph()
for ethread in longThreads:
    for previous, current in zip(ethread, ethread[1:]):
        G.add_edge(previous,current, attr=ethread )
pos = nx.drawing.layout.spring_layout(G)
nx.set_node_attributes(G, pos, 'pos')

In [200]:
import igraph as ig
g = ig.Graph.from_networkx(G)

In [201]:
import plotly.graph_objects as go
def plot_igraph_plotly(g, layout, title):
    """Takes a igraph graph object and plots it in plotly acccording to given layout"""
    Xe, Ye = [], []
    nr_vertices = len(g.vs)
    labels = g.vs.get_attribute_values('_nx_name')

    lay = g.layout(layout)
    position = {k: lay[k] for k in range(nr_vertices)}
    L = len(position)
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    E  = [e.tuple for e in g.es]
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]

    for edge in E:
        Xe += [position[edge[0]][0], position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

    fig = go.Figure(
        layout=go.Layout(title=f'{title}')
    )
    fig.add_trace(go.Scatter(x=Xe,
                    y=Ye,
                    mode='lines',
                    line=dict(color='rgb(210,210,210)', width=3),
                    hoverinfo='none'
                    ))
    fig.add_trace(go.Scatter(x=Xn,
                    y=Yn,
                    mode='markers',
                    name='bla',
                    marker=dict(symbol='circle-dot',
                                    size=6,
                                    color='#6175c1',    #'#DB4551',
                                    line=dict(color='rgb(50,50,50)', width=1)
                                    ),
                    text=labels,
                    hoverinfo='text',
                    opacity=0.8
                    ))
    fig.update_layout(showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    return fig

In [202]:
plot_igraph_plotly(g, layout='rt_circular', title=f'Thread Length over {3}')

In [36]:
import pickle
pickle.dump(threads_by_email, open('./data/threads_by_email.p', 'wb'))


In [227]:
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,0,0,Subject: Copy: City of Flint Drinking Water Is...,Staff_11_0000.txt,NaN,NaN,"[Brader; Valerie (GOV), Busch, Stephen (DEQ), ...",[],Copy: City of Flint Drinking Water Issues Loca...,CH-6S-DEQ-DIRECTOR,Update: This meeting has been confirmed for Mo...
1,1,1,Subject: City of Flint Drinking Water Issues L...,Staff_11_0001.txt,NaN,NaN,"[Brader; Valerie (GOV), Busch, Stephen (DEQ), ...",[],City of Flint Drinking Water Issues Location,CH-6S-DEQ-DIRECTOR,Update: This meeting has been confirmed for Mo...
2,2,2,NaN,Staff_11_0002.txt,NaN,NaN,[],[],NaN,NaN,NaN
3,3,2,"From: Wisniewski, Wendy (GOV) Sent: Wednesday;...",Staff_11_0002.txt,"Wisniewski, Wendy (GOV)","Wednesday; April 29, 2015 1.25 PM",[West; Samantha (GOV)],[],FW: Governor Snyder's Termination Letter,Flint EM Termination Letter pdf,NaN
4,4,2,"From: Wisniewski, Wendy (GOV) Sent: Wednesday,...",Staff_11_0002.txt,"Wisniewski, Wendy (GOV)","Wednesday, April 29, 2015 12.08 PM","[Stanton, Terry A. (Treasury), Storberg, Ann M...","[Koryzno, Edward (Treasury), Wisniewski, Wendy...",FW: Governor Snyder's Termination Letter,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30105,30105,16920,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1388.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."
30106,30106,16921,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1389.txt,NaN,NaN,[],[],NaN,City of Flint; Michigan Page 8 Printed on,4/2/2013
30107,30107,16922,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1390.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."
30108,30108,16923,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1391.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."


In [233]:
from nltk import sent_tokenize
import re
df = metadata.drop(metadata.loc[metadata.Body.isnull()].index)
df['sents'] = df.Body.apply(lambda x: [s for s in sent_tokenize(x) if (len(re.findall('\t',s))<2) and (len(s.split(' '))>4)])

In [235]:
sents = df.reset_index().drop(['From', 'Body', 'To', 'Cc', 'Subject', 'Attachment', 'Sent'], axis=1).explode('sents').reset_index(drop=True).rename(columns={'index':'org_idx'})
data = sents.sents.dropna()

In [236]:
from txtai.embeddings import Embeddings
embeddings = Embeddings({"path": "sentence-transformers/msmarco-bert-base-dot-v5"})

In [237]:
embeddings.index([(uid, text, None) for uid, text in enumerate(data)])

: 

: 

In [ ]:
uid = embeddings.search('water', limit=5)[4][0]
data[uid]

In [16]:
import pickle
import pandas as pd

with open('./data/threads_by_email.p', 'rb') as f:
    email_threads = pickle.load(f)

metadata = pd.read_csv('./data/metadata_for_threads.csv')

In [17]:
import networkx as nx
import igraph as ig

longThreads = []
for ethread in email_threads:
    if len(ethread) > 6:
        longThreads.append(ethread)

G = nx.DiGraph()
for ethread in longThreads:
    for previous, current in zip(ethread, ethread[1:]):
        G.add_edge(previous,current, attr=ethread )
pos = nx.drawing.layout.spring_layout(G)
nx.set_node_attributes(G, pos, 'pos')

g = ig.Graph.from_networkx(G)

In [29]:
g.vs[1]['_nx_name'].replace('<br>', ' ')

'Email from Klotz, Josephine (MSP) to Clayton, Stacie (GOV)  , concerning RE: Hollins/ Muchmore/Etue Call Today'

In [32]:
for t in email_threads:
    for e in t:
        if e == g.vs[1]['_nx_name'].replace('<br>', ''):
            print(e)

In [5]:
import pandas as pd
from ast import literal_eval

metadata = pd.read_csv('./data/lookup_data_for_threads.csv')
for col in ['To', 'Cc']:
    metadata[col] = metadata[col].apply(literal_eval)
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,0,0,Subject: Copy: City of Flint Drinking Water Is...,Staff_11_0000.txt,NaN,NaN,"[Brader; Valerie (GOV), Busch, Stephen (DEQ), ...",[],Copy: City of Flint Drinking Water Issues Loca...,CH-6S-DEQ-DIRECTOR,Update: This meeting has been confirmed for Mo...
1,1,1,Subject: City of Flint Drinking Water Issues L...,Staff_11_0001.txt,NaN,NaN,"[Brader; Valerie (GOV), Busch, Stephen (DEQ), ...",[],City of Flint Drinking Water Issues Location,CH-6S-DEQ-DIRECTOR,Update: This meeting has been confirmed for Mo...
2,2,2,NaN,Staff_11_0002.txt,NaN,NaN,[],[],NaN,NaN,NaN
3,3,2,"From: Wisniewski, Wendy (GOV) Sent: Wednesday;...",Staff_11_0002.txt,"Wisniewski, Wendy (GOV)","Wednesday; April 29, 2015 1.25 PM",[West; Samantha (GOV)],[],FW: Governor Snyder's Termination Letter,Flint EM Termination Letter pdf,NaN
4,4,2,"From: Wisniewski, Wendy (GOV) Sent: Wednesday,...",Staff_11_0002.txt,"Wisniewski, Wendy (GOV)","Wednesday, April 29, 2015 12.08 PM","[Stanton, Terry A. (Treasury), Storberg, Ann M...","[Koryzno, Edward (Treasury), Wisniewski, Wendy...",FW: Governor Snyder's Termination Letter,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30105,30105,16920,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1388.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."
30106,30106,16921,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1389.txt,NaN,NaN,[],[],NaN,City of Flint; Michigan Page 8 Printed on,4/2/2013
30107,30107,16922,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1390.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."
30108,30108,16923,"CITY COUNCIL Meeting Minutes Draft March 25, 2...",Staff_13_1391.txt,NaN,NaN,[],[],NaN,NaN,"ITY COUNCIL Meeting Minutes Draft March 25, 20..."


In [4]:
metadata.dropna(subset='Body')

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,0,0,Appointment,Staff_17_0000.txt,NaN,NaN,[],[],NaN,NaN,ppointment
1,1,0,"From: Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHA...",Staff_17_0000.txt,"Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHANGE AD...","1/13/2015 3.32:19 PM To: Posthumus, Dick (GOV)...","[Pleyte, Beth (Treasury) [O-MIGOV/OU-EXTernal ...",[],Copy: Mtg Flint Water Issues/other Concerns Lo...,1/28/2015 2.00.00 PM End: 1/28/2015 3.00.00 PM...,Attending= Jerry Ambrose Mayor Wa] Ting Tom Sa...
3,3,1,"From: Brader;, Valerie (GOV) Sent: Tuesday, Se...",Staff_17_0001.txt,NaN,"Tuesday, September 23, 2014 2.47 PM To: Ackerm...",[],[],NaN,NaN,Uh????\n
4,4,1,"From: West; Samantha (GOV) Sent: Tuesday, Sept...",Staff_17_0001.txt,NaN,"Tuesday, September 23, 2014 2.43 PM","[Brader , Valerie (GOV)]",[],Phone Msg: Senator Ananich,NaN,Called this afternoon: He was speaking with Ro...
6,6,2,"From: Lange; Michelle (GOV) Sent: Tuesday, Oct...",Staff_17_0002.txt,NaN,"Tuesday, October 06, 2015 10.01 AM To: Posthum...",[],[],NaN,NaN,Good morning-\nHere is the latest from comms:\...
...,...,...,...,...,...,...,...,...,...,...,...
30103,30103,16922,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1358.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 12.25 PM To: 'Egan, ...",[],[],NaN,NaN,"Paul, | don'tknow if someone was able to get y..."
30104,30104,16922,"From: Egan, Paul [meiltopegan@treepress cqm] S...",Staff_12_1358.txt,"Egan, Paul [meiltopegan@treepress cqm]","Tuesday, January 05, 2016 11:56 AM","[Murray, David (GOV) <Mu.axpl@nchigengov?]",[],US Attorney's Office Flint,NaN,Dave can you give me any comment on US Attorne...
30106,30106,16923,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1359.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 12.42 PM","[Egan, Paul (pegan@freepresscom)]",[],Correct statement,NaN,"Paul, accidently sentyouthe outdated statement..."
30108,30108,16924,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1360.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 1.09 PM","['Hollins, Harvey (GOV)]",[],RE: Flint Talking Point,NaN,"Hello Harvey, after consulting with Jim Redfor..."


In [ ]:
import pandas as pd
from ast import literal_eval

metadata = pd.read_csv('./data/metadata_for_threads.csv').dropna(subset='Body').reset_index(drop=True)
for col in ['To', 'Cc']:
    metadata[col] = metadata[col].apply(literal_eval)
metadata

,Unnamed: 0,thread_index,parsed_email,image_lookup,From,Sent,To,Cc,Subject,Attachment,Body
0,0,0,Appointment,Staff_17_0000.txt,NaN,NaN,[],[],NaN,NaN,ppointment
1,1,0,"From: Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHA...",Staff_17_0000.txt,"Rospond, Laurie (GOV) [/O-MIGOV/OU-EXCHANGE AD...","1/13/2015 3.32:19 PM To: Posthumus, Dick (GOV)...","[Pleyte, Beth (Treasury) [O-MIGOV/OU-EXTernal ...",[],Copy: Mtg Flint Water Issues/other Concerns Lo...,1/28/2015 2.00.00 PM End: 1/28/2015 3.00.00 PM...,Attending= Jerry Ambrose Mayor Wa] Ting Tom Sa...
2,3,1,"From: Brader;, Valerie (GOV) Sent: Tuesday, Se...",Staff_17_0001.txt,NaN,"Tuesday, September 23, 2014 2.47 PM To: Ackerm...",[],[],NaN,NaN,Uh????\n
3,4,1,"From: West; Samantha (GOV) Sent: Tuesday, Sept...",Staff_17_0001.txt,NaN,"Tuesday, September 23, 2014 2.43 PM","[Brader , Valerie (GOV)]",[],Phone Msg: Senator Ananich,NaN,Called this afternoon: He was speaking with Ro...
4,6,2,"From: Lange; Michelle (GOV) Sent: Tuesday, Oct...",Staff_17_0002.txt,NaN,"Tuesday, October 06, 2015 10.01 AM To: Posthum...",[],[],NaN,NaN,Good morning-\nHere is the latest from comms:\...
...,...,...,...,...,...,...,...,...,...,...,...
21091,30103,16922,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1358.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 12.25 PM To: 'Egan, ...",[],[],NaN,NaN,"Paul, | don'tknow if someone was able to get y..."
21092,30104,16922,"From: Egan, Paul [meiltopegan@treepress cqm] S...",Staff_12_1358.txt,"Egan, Paul [meiltopegan@treepress cqm]","Tuesday, January 05, 2016 11:56 AM","[Murray, David (GOV) <Mu.axpl@nchigengov?]",[],US Attorney's Office Flint,NaN,Dave can you give me any comment on US Attorne...
21093,30106,16923,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1359.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 12.42 PM","[Egan, Paul (pegan@freepresscom)]",[],Correct statement,NaN,"Paul, accidently sentyouthe outdated statement..."
21094,30108,16924,"From: Murray, David (GOV) Sent: Tuesday, Janua...",Staff_12_1360.txt,"Murray, David (GOV)","Tuesday, January 05, 2016 1.09 PM","['Hollins, Harvey (GOV)]",[],RE: Flint Talking Point,NaN,"Hello Harvey, after consulting with Jim Redfor..."


In [ ]:
input_texts = list(metadata['Body'])

In [ ]:
from txtai.embeddings import Embeddings
embeddings = Embeddings()
embeddings.load('./data/fl_index622023')

In [ ]:
s = embeddings.search('corruption', limit=10)

In [ ]:
input_texts[s[6][0]]

'This catastrophe began with the governor\'s appointed emergency manager; and as much as Governor Snyder would like to pretend; this is not a matter of negotiating at the boardroom table. This is a situation where you pay for your mistakes and put those responsible on the chopping block:\n"That is why we are calling for the immediate firing of the governor\'s handpicked emergency manager for Flint; Darnell Earley: Earley was in charge when the decision was made to switch the city to unsafe drinking water. If poisoning children wasn\'t bad enough, after Earley failed the citizens of Flint;, the governor inexplicably rewarded him with another appointment emergency manager for Detroit Public Schools S where he is now responsible for the education of nearly 50,000 students:\n"The children of Flint and Detroit are being robbed of what most of us take for granted S safe drinking water and a quality education. The wholesale theft of our children\'s futures must end now: That begins with the r

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)